In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

2023-05-24 14:00:06.036129: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-24 14:00:06.316457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 14:00:08.013029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
IMG_SIZE = (224, 224)

train_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

Found 25000 files belonging to 2 classes.
Using 17500 files for training.


2023-05-24 14:01:35.774314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-24 14:01:35.865324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-24 14:01:35.865418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-24 14:01:35.867658: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-24 14:01:35.867708: I tensorflow/compile

Found 25000 files belonging to 2 classes.
Using 7500 files for validation.


In [3]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip(),
        keras.layers.RandomRotation(0.2),
        keras.layers.RandomContrast(0.5),
        keras.layers.RandomBrightness(0.5)
    ]
)

In [4]:
plt.figure(1)

for images, labels in train_ds.take(1):
    or_img = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        augm_img = data_augmentation(
            tf.expand_dims(or_img, 0),
            training=True)
        plt.imshow(augm_img[0].numpy().astype('uint8'))
        plt.axis('off')

2023-05-24 14:03:23.113058: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [17500]
	 [[{{node Placeholder/_4}}]]
2023-05-24 14:03:23.114132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [17500]
	 [[{{node Placeholder/_4}}]]
2023-05-24 14:03:23.292744: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : RESOURCE_EXHAUSTED: data/cats_dogs/dogs/dog.11650.jpg; Cannot allocate memory
2023-05-24 14:03:23.293069: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/tas

ResourceExhaustedError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} data/cats_dogs/cats/cat.10378.jpg; Cannot allocate memory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext]

<Figure size 640x480 with 0 Axes>

In [5]:
base_model = keras.applications.MobileNetV3Small(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)

base_model.trainable = False

# sequential
model = keras.Sequential()
model.add(keras.layers.Lambda(keras.applications.mobilenet_v3.preprocess_input, input_shape=(224, 224, 3)))
model.add(data_augmentation)
model.add(base_model)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1))

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.Recall()
    ]
)

model.fit(train_ds, epochs=5, validation_data=val_ds)

4334752/4334752 [==============================] - 0s 0us/step
Epoch 1/5


2023-05-24 14:12:34.817821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [17500]
	 [[{{node Placeholder/_4}}]]
2023-05-24 14:12:34.818044: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [17500]
	 [[{{node Placeholder/_0}}]]
2023-05-24 14:12:47.607388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8901
2023-05-24 14:12:55.026413: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-24 14:12:55.556878: I tensorflow/compiler/xl

1094/1094 [==============================] - ETA: 0s - loss: 7.5525 - binary_accuracy: 0.5001

2023-05-24 14:13:25.965914: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [7500]
	 [[{{node Placeholder/_4}}]]
2023-05-24 14:13:25.966204: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [7500]
	 [[{{node Placeholder/_0}}]]


1094/1094 [==============================] - 60s 35ms/step - loss: 7.5525 - binary_accuracy: 0.5001 - val_loss: 7.5674 - val_binary_accuracy: 0.5027
Epoch 2/5
1094/1094 [==============================] - 37s 33ms/step - loss: 7.6491 - binary_accuracy: 0.4979 - val_loss: 7.5762 - val_binary_accuracy: 0.5027
Epoch 3/5
1092/1094 [============================>.] - ETA: 0s - loss: 7.6512 - binary_accuracy: 0.4981

KeyboardInterrupt: 